In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from Thinet import Thinet
from DataManager import load_data_from_file

In [3]:
data = load_data_from_file(r'./data/cifar10_features_5000.npz')

Conv1 weights:  (5, 5, 3, 64)
Conv2 input:  (5000, 12, 12, 64)
Conv2 weights:  (5, 5, 64, 64)
Conv2 output:  (5000, 12, 12, 64)
Loaded 5000 samples


In [9]:
input_map  = data['conv2_input']    # (5000,12,12,64)
weights    = data['conv2_weights']  # (5,5,3,64)
output_map = data['conv2_output']   # (5000,12,12,64)
#rates      = [0.375, 0.25, 0.125, 0.0625]
rates      = [0.75, 0.5]
num_points = 50

for rate in rates:
    thinet = Thinet(input_map, weights, output_map, rate)

    thinet.compress(num_points)

    output_file = r'./data/thinet_compression_%d.npy' % (rate * 64)
    thinet.save_data_to_file(output_file)
    print("Saved to: %s" % output_file)

Filters prunned: 1 in 3.755 [s]
Filters prunned: 2 in 10.885 [s]
Filters prunned: 3 in 21.307 [s]
Filters prunned: 4 in 34.992 [s]
Filters prunned: 5 in 51.773 [s]
Filters prunned: 6 in 71.516 [s]
Filters prunned: 7 in 94.236 [s]
Filters prunned: 8 in 119.808 [s]
Filters prunned: 9 in 148.324 [s]
Filters prunned: 10 in 179.512 [s]
Filters prunned: 11 in 213.457 [s]
Filters prunned: 12 in 249.911 [s]
Filters prunned: 13 in 288.801 [s]
Filters prunned: 14 in 330.053 [s]
Filters prunned: 15 in 373.766 [s]
Filters prunned: 16 in 419.824 [s]
Saved to: ./data/thinet_compression_48.npy
Filters prunned: 1 in 3.763 [s]
Filters prunned: 2 in 10.900 [s]
Filters prunned: 3 in 21.324 [s]
Filters prunned: 4 in 34.995 [s]
Filters prunned: 5 in 51.709 [s]
Filters prunned: 6 in 71.553 [s]
Filters prunned: 7 in 94.260 [s]
Filters prunned: 8 in 119.961 [s]
Filters prunned: 9 in 148.502 [s]
Filters prunned: 10 in 180.222 [s]
Filters prunned: 11 in 215.025 [s]
Filters prunned: 12 in 251.619 [s]
Filters pru

In [6]:
data_dict = np.load('./data/thinet_compression_4.npy').item()
print("Filters to keep: ", data_dict['keep_filters'])
print("Filters to prune: ", data_dict['remove_filters'])
print(data_dict['weights_wo_reconst'].shape)
print(data_dict['weights_reconst'].shape)

Filters to keep:  [35, 50, 57, 58]
Filters to prune:  [21, 53, 3, 17, 15, 14, 32, 19, 47, 8, 39, 16, 51, 49, 34, 31, 38, 30, 29, 46, 37, 24, 9, 4, 44, 54, 62, 13, 12, 61, 22, 5, 48, 63, 43, 26, 23, 28, 25, 36, 10, 6, 18, 20, 52, 42, 7, 1, 2, 59, 11, 56, 0, 27, 45, 60, 33, 40, 55, 41]
(5, 5, 4, 64)
(5, 5, 4, 64)
